In [33]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('/home/jupyter-23521027/refresh-bert/data/sbert/indosbert_dense_trained', device='cuda:1')

In [39]:
import os
import json
import pandas as pd

folder_path = '/home/jupyter-23521027/refresh-bert/data/liputan6_data/canonical/train' 

file_list = os.listdir(folder_path)
json_data_list = []
file_list = os.listdir(folder_path)

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name) 

    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            json_data = json.load(file)
            json_data_list.append(json_data)

df = pd.DataFrame(json_data_list)
df['sentences'] = df['clean_article'].apply(lambda x: [' '.join(sentence) for sentence in x])
df.tail(1)

,id,url,clean_article,clean_summary,extractive_summary,sentences
193882,65953,https://www.liputan6.com/news/read/65953/ditem...,"[[Liputan6, ., com, ,, Langkat, :, Tim, Search...","[[Tim, SAR, kembali, menemukan, satu, jenazah,...","[1, 2, 3]","[Liputan6 . com , Langkat : Tim Search and Res..."


In [40]:
file_path = "/home/jupyter-23521027/refresh-bert/data/preprocessed-input-directory/liputan6.training.sbert"

existing_ids = []
with open(file_path, 'r') as file:
    lines = file.readlines()

    for line in lines:
        line = line.strip()

        if line and ' ' not in line:
            existing_ids.append(line.split("-")[1])
            
print(existing_ids[:5])
print(len(existing_ids))

['48951', '124973', '298677', '40569', '242869']
5000


In [41]:
print(existing_ids[-5:])


['161517', '252163', '266451', '138172', '268940']


In [ ]:
from datetime import datetime

MAX_DOC_LENGTH = 100
MAX_SENT_EMBEDDING_LENGTH = 250
DATASET_SIZE = 999999

now = datetime.now()
file = open(file_path , 'a')
file.write("\n")

for i, row in df.iterrows():
    if str(row['id']) in existing_ids:
        continue

    file.write(f"liputan6-{str(row['id'])}\n")

    for sent_txt in row["sentences"][:MAX_DOC_LENGTH]:
        sent_embedding = model.encode(sent_txt, convert_to_numpy=True).tolist()[:MAX_SENT_EMBEDDING_LENGTH]

        for element in sent_embedding:
            file.write(str(element) + " ")
            
        file.write("\n")
    
    if (i + 1) % 100 == 0:
        print(f"processing {i + 1} with time: {datetime.now() - now}")
        now = datetime.now()
    
    if (i + 1) == DATASET_SIZE:
        break
    
    file.write("\n")
        
file.close()

processing 5100 with time: 0:00:20.144996
processing 5200 with time: 0:00:19.026860
processing 5300 with time: 0:00:17.651931
processing 5400 with time: 0:00:18.416491
processing 5500 with time: 0:00:20.466080
processing 5600 with time: 0:00:20.985286
processing 5700 with time: 0:00:19.946282
processing 5800 with time: 0:00:17.661424
processing 5900 with time: 0:00:20.000339
processing 6000 with time: 0:00:20.257800
processing 6100 with time: 0:00:18.709670
processing 6200 with time: 0:00:20.520834
processing 6300 with time: 0:00:19.070736
processing 6400 with time: 0:00:18.714344
processing 6500 with time: 0:00:20.678696
processing 6600 with time: 0:00:20.323049
processing 6700 with time: 0:00:20.207883
processing 6800 with time: 0:00:20.198178
processing 6900 with time: 0:00:21.667910
processing 7000 with time: 0:00:21.162581
processing 7100 with time: 0:00:20.396082
processing 7200 with time: 0:00:19.291939
processing 7300 with time: 0:00:19.515699
processing 7400 with time: 0:00:19

In [ ]:
!cat /home/jupyter-23521027/refresh-bert/data/preprocessed-input-directory/liputan6.test.sbert | tail -c 100000